In [2]:
"""
base class

A node has a value and 3 pointers - 
left, right and parent.
"""

class Node:
    def __init__(self,value):
        self._value = value
        self._left = None
        self._right = None
        self._parent = None
    
    @property
    def value(self):
        return self._value
    
    @property
    def left(self):
        return self._left
    
    @left.setter
    def left(self,l):
        self._left = l
    
    @property
    def right(self):
        return self._right
    
    @right.setter
    def right(self,r):
        self._right = r
        
    @property
    def parent(self):
        return self._parent
    
    @parent.setter
    def parent(self,p):
        self._parent = p

In [3]:
"""
binary search tree


Each vertex has only up to 2 children. 

Complexity:
Basic Ops - is proportional to the height of tree, thus O(h)
in worst cases it is O(N)
in average it is O(lgN)

"""

class BST:
    def __init__(self):
        self._root = None
    
    @property
    def root(self):
        return self._root
    
    @root.setter
    def root(self, root):
        self._root = root
    
    # print values of all nodes
    def inorder_walk(self, node):
        if node != None:
            self.inorder_walk(node.left)
            print(node.value)
            self.inorder_walk(node.right)
            
            
    # query a node with specific value
    def search(self, value):
        node = self._root
        while node != None and node.value != value:
            if node.value < value:
                node = node.right
            else:
                node = node.left
        return node
    
    
    # find node of min value
    def minimum(self, node):
        while node and node.left != None:
            node = node.left
        return node
    
    
    # find node of max value
    def maximum(self, node):
        while node and node.right != None:
            node = node.right
        return node
    
    
    # find successore in sorted order
    def successor(self, node):
        if node.right != None:
            return self.minimum(node.right)
        else:
            p = node.parent
            while p != None and p.right == node:
                node = p
                p = p.parent
            return p
    
    
    # find predecessor in sorted order
    def predecessor(self, node):
        if node.left != None:
            return self.maximum(node.right)
        else:
            p = node.parent
            while p != None and p.left == node:
                node = p
                p = p.parent
            return p        

        
    # insert a new node
    def insert(self, node):
        value = node.value
        n = self._root
        parent = None
        while n != None:
            parent = n
            if n.value > value:
                n = n.left
            else:
                n = n.right
        node.parent = parent
        if parent == None:
            self._root = node
        else:
            if parent.value > value:
                parent.left = node
            else:
                parent.right = node
     

    # when v is u's left or right children
    # by transplanting, u's parent's (left/right) child become v, u's parent becomes v's parent
    # but, u's another children is not touched
    def _transplant(self, u, v):
        # u is root
        if u.parent == None:
            self.root = v
        # u is its parent's left children
        elif u.parent.left == u:
            u.parent.left = v
        # u is its parent's right children
        else:
            u.parent.right = v
            
        if v != None:
            v.parent = u.parent
            
    
    # del a node of value 
    def delete(self, value):
        node = self.search(value)
        if node == None:
            return

        # case 1: node has no children, aka leaf node
        # delete right away
        if node.left == None and node.right == None:
            self._transplant(node, None)
            return

        # case 2: node is an internal/root node with one children
        # do one transplant
        if node.left == None:
            self._transplant(node, node.right)
            return
        elif node.right == None:
            self._transplant(node, node.left)  
            return
        
        # case 3: node is an internal/root node with two children
        # replace node with its successor
        suc = self.successor(node)
        # successor is node's children
        if suc.parent == node:
            self._transplant(node, suc)
            suc.left = node.left
            node.left.parent = suc 
        # successor is not node's right children, but in the subtree
        else:
            self._transplant(suc, suc.right)
            suc.right = node.right
            suc.right.parent = suc
            self._transplant(node.suc)
            suc.left = node.left
            node.left.parent = suc

In [82]:
# test bst
bst = BST()
print("insert 0-5")
for i in [2,3,5,4,0,1]: 
    bst.insert(Node(i))
bst.inorder_walk(bst.root)
print("min {0}".format(bst.minimum(bst.root).value))
print("max {0}".format(bst.maximum(bst.root).value))
four = bst.search(4)
print("predecessor of {0} is {1}".format(four.value, bst.predecessor(four).value))
print("successor of {0} is {1}".format(four.value, bst.successor(four).value))
print("delete 4")
bst.delete(4)
bst.inorder_walk(bst.root)
print("delete 2")
bst.delete(2)
bst.inorder_walk(bst.root)
print("insert 2 back")
bst.insert(Node(2))
bst.inorder_walk(bst.root)

insert 0-5
0
1
2
3
4
5
min 0
max 5
predecessor of 4 is 3
successor of 4 is 5
delete 4
0
1
2
3
5
delete 2
0
1
3
5
insert 2 back
0
1
2
3
5


In [87]:
"""
red-black tree

properties:
1. Every node is either red or black;
2. Root node is black;
3. Every Nil leaf is black;
4. A red node has two black children nodes;
5. Every path from root to leaf has same number of black nodes (black-height).

1 to 1 correspondence to a 2-3 tree
"""

# empty node
class Nil(Node):
    def __init__(self):
        Node.__init__(self, None)
        self._color = 'B'
        
    @property
    def color(self):
        return self._color

    def __str__(self,level=0):
        return "\t"*level+"Nil"+"\n"
    
    
# tree node class       
class RBNode(Node):
    
    def __init__(self,value):
        Node.__init__(self, value)
        self._left = Nil()
        self._right = Nil()
        self._parent = Nil()
        self._color = None

    @property
    def color(self):
        return self._color

    @color.setter
    def color(self,color):
        self._color = color
    
    # print self, left node and right node
    def __str__(self, level=0):
        ret = "\t"*level+repr(self._value)+ " " + repr(self._color)+"\n"
        ret += self._left.__str__(level+1)
        ret += self._right.__str__(level+1)
        return ret
        
# tree class
class RBTree(BST):
    
    def __init__(self):
        self._root = Nil()
    
    # print all nodes 
    def __str__(self):
        return self._root.__str__(0)
        
#     def inorder_walk(self, node, level):
#         if not isinstance(node,Nil):
#             self.inorder_walk(node.left, level+1)
#             print("value={0} color={1} @ level {2}".format(node.value, node.color, level))
#             self.inorder_walk(node.right, level+1)
            
    # rotation op to change pointer structure
    def _left_rotate(self, node_x):
        node_y = node_x.right
        node_x.right = node_y.left
        if not isinstance(node_y.left, Nil):
            node_y.left.parent = node_x
        node_y.parent = node_x.parent
        if isinstance(node_x.parent, Nil):
            self._root = node_y
        elif node_x == node_x.parent.left:
            node_x.parent.left = node_y
        else:
            node_x.parent.right = node_y
        node_y.left = node_x
        node_x.parent = node_y
        
    # symmetric to _left_rotate
    # where node_x => node_y, node_y => node_x
    # .left => .right and .right => .left
    def _right_rotate(self, node_y):
        node_x = node_y.left
        node_y.left = node_x.right
        if not isinstance(node_x.right, Nil):
            node_x.right.parent = node_y
        node_x.parent = node_y.parent
        if isinstance(node_y.parent, Nil):
            self._root = node_x
        elif node_y == node_y.parent.right:
            node_y.parent.right = node_x
        else:
            node_y.parent.left = node_x
        node_x.right = node_y
        node_y.parent = node_x
    
    
    def insert(self, rbnode):
        # same as BST insert
        value = rbnode.value
        n = self._root
        parent = Nil()
        while not isinstance(n, Nil):
            parent = n
            if n.value > value:
                n = n.left
            else:
                n = n.right
        rbnode.parent = parent
        if isinstance(parent, Nil):
            self._root = rbnode
        else:
            if parent.value > value:
                parent.left = rbnode
            else:
                parent.right = rbnode

        # additional
        rbnode.left = Nil()
        rbnode.right = Nil()
        rbnode.color = 'R'
        
        # fix potential property 2 violation
        # where rbnode's parent is a red node
        n = rbnode
        while not isinstance(n.parent,Nil) and n.parent.color == 'R':
            if not isinstance(n.parent.parent, Nil):
                # parent is left child, red uncle
                # or parent is right child, red uncle
                if n.parent == n.parent.parent.left:
                    if not isinstance(n.parent.parent.right,Nil) and n.parent.parent.right.color == 'R':
                        print('case 1')
                        n.parent.parent.color = 'R'                
                        n.parent.parent.left.color = 'B'
                        n.parent.parent.right.color = 'B'
                        n = n.parent.parent
                        # continue while loop
                    else:
                        if n == n.parent.right:
                            print('case 2')
                            n = n.parent
                            self._left_rotate(n)
                        else:
                            print('case 3')
                        n.parent.color = 'B'
                        n.parent.parent.color = 'R'
                        self._right_rotate(n.parent.parent)
                        # stop loop
                        
                # symmetric as the above "if"
                elif n.parent == n.parent.parent.right:
                    if not isinstance(n.parent.parent.left,Nil) and n.parent.parent.left.color == 'R':
                        print('case 4')
                        n.parent.parent.color = 'R'                
                        n.parent.parent.left.color = 'B'
                        n.parent.parent.right.color = 'B'
                        n = n.parent.parent
                    else:
                        if n == n.parent.left:
                            print('case 5')
                            n = n.parent
                            self._right_rotate(n)
                        else:
                            print('case 6')
                        n.parent.color = 'B'
                        n.parent.parent.color = 'R'
                        self._left_rotate(n.parent.parent)
                        
        # fix potential property 4 violation
        # where rbnode is root                
        self._root.color = 'B'
        
        
        

In [88]:
rbt = RBTree()

for i in [2,3,5,4,0,6,1,9,7,8]:
    print("after inserting {0}".format(i))
    rbt.insert(RBNode(i))
    print(rbt)

after inserting 2
2 'B'
	Nil
	Nil

after inserting 3
2 'B'
	Nil
	3 'R'
		Nil
		Nil

after inserting 5
case 6
3 'B'
	2 'R'
		Nil
		Nil
	5 'R'
		Nil
		Nil

after inserting 4
case 4
3 'B'
	2 'B'
		Nil
		Nil
	5 'B'
		4 'R'
			Nil
			Nil
		Nil

after inserting 0
3 'B'
	2 'B'
		0 'R'
			Nil
			Nil
		Nil
	5 'B'
		4 'R'
			Nil
			Nil
		Nil

after inserting 6
3 'B'
	2 'B'
		0 'R'
			Nil
			Nil
		Nil
	5 'B'
		4 'R'
			Nil
			Nil
		6 'R'
			Nil
			Nil

after inserting 1
case 2
3 'B'
	1 'B'
		0 'R'
			Nil
			Nil
		2 'R'
			Nil
			Nil
	5 'B'
		4 'R'
			Nil
			Nil
		6 'R'
			Nil
			Nil

after inserting 9
case 4
3 'B'
	1 'B'
		0 'R'
			Nil
			Nil
		2 'R'
			Nil
			Nil
	5 'R'
		4 'B'
			Nil
			Nil
		6 'B'
			Nil
			9 'R'
				Nil
				Nil

after inserting 7
case 5
3 'B'
	1 'B'
		0 'R'
			Nil
			Nil
		2 'R'
			Nil
			Nil
	5 'R'
		4 'B'
			Nil
			Nil
		7 'B'
			6 'R'
				Nil
				Nil
			9 'R'
				Nil
				Nil

after inserting 8
case 4
case 6
5 'B'
	3 'R'
		1 'B'
			0 'R'
				Nil
				Nil
			2 'R'
				Nil
			